## Installation of TwelveLabs SDKs

In [1]:
!pip install -U -q twelvelabs

## Configure your API key
Add API Key as a env variable. Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.io/?_gl=1*1rx2bxa*_ga*MTUzMTIzMTI3MC4xNzI3OTAxMzE2*_ga_END0TB2RFD*MTc0MDE4NTM2NS4yNC4xLjE3NDAxODY1MjkuMC4wLjA.). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

### Create an AWS Secrets Manager secret 

Follow steps here to [create a secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) in AWS Secrets Manager. For example, name the secret as `TL_API_Key`.

Note down the ARN or name of the secret (`TL_API_Key`) to retrieve. To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

We will be using this value for the `SecretId` in the code block below.

In [2]:
import boto3
import json
secrets_manager_client=boto3.client("secretsmanager")
API_secret=secrets_manager_client.get_secret_value(
    SecretId="TL_API_KEY"
)
TL_API_KEY=json.loads(API_secret["SecretString"])["TL_API_Key"]

## Using video data

For our demo, we will work on this video: [Robin bird forest free stock video via Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/). We have downloaded the video locally and uploaded it to the S3 bucket above.

This demo depends upon some video data. To use is, we will download an mp4 file and upload it to an Amazon S3 bucket.

1. Click on this link containing the [Robin bird forest free stock video via Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/)
1. Download the 21723-320725678_small.mp4 file.
1. Create an S3 bucket if you don't have one already. Follow the steps in the [Creating a bucket doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/create-bucket-overview.html). Note down the name of the bucket (Eg., `MYS3BUCKET`).
1. Upload the 21723-320725678_small.mp4 video file to the S3 bucket created in the step above by following the stesp in the [Uploading objects doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/upload-objects.html). Note down the name of the object (Eg., `21723-320725678_small.mp4`)


In [5]:
s3_client=boto3.client("s3")
video_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='21723-320725678_small.mp4', Filename='robin-bird.mp4')

## Generate Embeddings
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.
Learn more in the [docs](https://docs.twelvelabs.io/docs/create-video-embeddings)

In [6]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)

task = twelvelabs_client.embed.task.create(
    model_name="Marengo-retrieval-2.7",
    video_file="robin-bird.mp4"
)
print(
    f"Created task: id={task.id} engine_name={task.model_name} status={task.status}"
)

def on_task_update(task: EmbeddingsTask):
    print(f"  Status={task.status}")

status = task.wait_for_done(
    sleep_interval=2,
    callback=on_task_update
)
print(f"Embedding done: {status}")

task = task.retrieve()
if task.video_embedding is not None and task.video_embedding.segments is not None:
    print_segments(task.video_embedding.segments)


Created task: id=67ca93a989d8a564e80dc3ba engine_name=Marengo-retrieval-2.7 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=ready
Embedding done: ready
  embedding_scope=clip start_offset_sec=0.0 end_offset_sec=6.0
  embeddings: [0.022429451, 0.00040668788, -0.01825908, -0.005862708, -0.03371106, -6.357456e-05, -0.015320076, -0.042556215, -0.02782445, -0.00019097517, 0.03258314, -0.0061399476, -0.00049206393, 0.035632476, 0.028209884, 0.02875258, -0.035486065, -0.11288028, -0.040782217, -0.0359422, 0.015908664, -0.021092793, 0.016303983, 0.06351931, -0.03370151, -0.059598114, 0.034242813, 0.013847672, -0.0077595757, 0.02485134, -0.00448969, 0.016366754, -0.015583887, 0.040098537, 0.009583407, -0.0028939361, 0.019334463, 0.028072173, -0.025249695, -0.023403516, -0.010630837, 0.02070483, 0.026287371, -0.026809538, 0.024101667, 0.02

## Create OpenSearch Domain

If you have an OpenSearch Service domain already, you can use the connection details for that domain. If you want to create a new domain, follow the steps in the [docs](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/createupdatedomains.html).

Follow [Operational best practices for Amazon OpenSearch Service](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/bp.html)

In [7]:
!pip install opensearch-py
!pip install botocore
!pip install requests-aws4auth

  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached Events-0.5-py3-none-any.whl (6.8 kB)
  Using cached requests_aws4auth-1.3.1-py3-none-any.whl.metadata (18 kB)
Using cached requests_aws4auth-1.3.1-py3-none-any.whl (24 kB)


## Getting OpenSearch Domain URL
Update your domain endpoint in the `host` section below.

You can find the domain endpoint under 'General Information' within details of domain created earlier.

Example: https://search-mydomain-1a2a3a4a5a6a7a8a9a0a9a8a7a.us-east-1.es.amazonaws.com

[Create an AWS Secrets Manager secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) for the Amazon OpenSearch username and password.

Note down the ARN or name of the secret (Eg., `AOS_password`, `AOS_username`) to retrieve . To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

In [8]:
import boto3
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from requests_aws4auth import AWS4Auth
from requests.auth import HTTPBasicAuth

# OpenSearch connection configuration
# host = 'your-host.aos.us-east-1.on.aws'
host = 'your-host.aos.us-east-1.on.aws'
port = 443  # Default HTTPS port

# Get OpenSearch username secret from Secrets Manager
opensearch_username=secrets_manager_client.get_secret_value(
    SecretId="AOS_username"
)
opensearch_username_string=json.loads(opensearch_username["SecretString"])["AOS_username"]

# Get OpenSearch password secret from Secrets Manager
opensearch_password = secrets_manager_client.get_secret_value(
    SecretId="AOS_password"
)
opensearch_password_string=json.loads(opensearch_password["SecretString"])["AOS_password"]

auth=(opensearch_username_string, opensearch_password_string)

# Create the client configuration
client_aos = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test the connection
try:
    # Get cluster information
    cluster_info = client_aos.info()
    print("Successfully connected to OpenSearch")
    print(f"Cluster info: {cluster_info}")
except Exception as e:
    print(f"Connection failed: {str(e)}")


# Define the index configuration

index_name = 'twelve_labs_index'
new_vector_index_definition = {
    "settings":
    {
        "index":
        {
            "knn": "true",
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    },
    "mappings":
    {
        "properties":
        {
            "embedding_field":
            {
                "type": "knn_vector",
                "dimension": 1024
            }
        }
    }
}

client_aos.indices.create(index="twelve_labs_index",body=new_vector_index_definition,ignore=400)
client_aos.indices.get(index="twelve_labs_index")

Successfully connected to OpenSearch
Cluster info: {'name': '7342fd21b4ca0df05ba9560a4b0847f2', 'cluster_name': '455169824512:new-domain', 'cluster_uuid': 'Gdf4gtsrSmOqi9s6gK4nIg', 'version': {'distribution': 'opensearch', 'number': '2.15.0', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2024-10-23T17:37:00.962018205Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


{'twelve_labs_index': {'aliases': {},
  'mappings': {'properties': {'embedding_field': {'type': 'knn_vector',
     'dimension': 1024}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'twelve_labs_index',
    'knn': 'true',
    'creation_date': '1734647018836',
    'number_of_replicas': '0',
    'uuid': '8azNxqCARBSg4E-jfrhtXQ',
    'version': {'created': '136367827'}}}}}

## Upsert the video embeddings into OpenSearch

In [9]:
if task.video_embedding is not None and task.video_embedding.segments is not None:
    embeddings_doc =task.video_embedding.segments
    # print_segments(embeddings_doc)

for doc_id, elt in enumerate(embeddings_doc):
    print(doc_id, elt.embeddings_float)
    document = {
        'embedding_field': elt.embeddings_float
    }
    try:
        response=client_aos.index(
            index = index_name,
            body = document,
            id = doc_id,
            refresh = True
        )
    except Exception as e:
        print(f"Error indexing document: {e}")

print ("Finished indexing of the data")

0 [0.022429451, 0.00040668788, -0.01825908, -0.005862708, -0.03371106, -6.357456e-05, -0.015320076, -0.042556215, -0.02782445, -0.00019097517, 0.03258314, -0.0061399476, -0.00049206393, 0.035632476, 0.028209884, 0.02875258, -0.035486065, -0.11288028, -0.040782217, -0.0359422, 0.015908664, -0.021092793, 0.016303983, 0.06351931, -0.03370151, -0.059598114, 0.034242813, 0.013847672, -0.0077595757, 0.02485134, -0.00448969, 0.016366754, -0.015583887, 0.040098537, 0.009583407, -0.0028939361, 0.019334463, 0.028072173, -0.025249695, -0.023403516, -0.010630837, 0.02070483, 0.026287371, -0.026809538, 0.024101667, 0.024479926, -0.015702814, 0.01550865, -0.016303834, 0.01995538, 0.019346705, 0.0451287, -0.022148034, 0.051220857, 0.058865245, -0.03188367, -0.03598581, 0.0017878609, -0.0062338635, 0.03417282, 0.013313552, -0.0005611755, 0.019141965, -0.038943443, -0.020611687, -0.014808893, 0.03933287, 0.0011992354, -0.025857447, 0.033376064, 0.043092463, 0.049742587, -0.01883242, 0.037969235, -0.047

## Create Text Embeddings and Perform Text-to-Video Search

In [10]:
def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

# Create text embeddings
text_res = twelvelabs_client.embed.create(
  model_name="Marengo-retrieval-2.7",
  text="Bird eating food",
)

print("Created a text embedding")
print(f" Model: {text_res.model_name}")

if text_res.text_embedding is not None and text_res.text_embedding.segments is not None:
        print_segments(text_res.text_embedding.segments)

Created a text embedding
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=None end_offset_sec=None
  embeddings: [0.036621094, -0.0061950684, 0.018188477, -0.041992188, -0.024658203, 0.018432617, -0.01953125, 0.007507324, -0.032714844, -0.017700195, 0.014465332, -0.020019531, 0.007659912, 0.018676758, 0.029785156, -0.01965332, 0.0010299683, -0.022827148, 0.0022277832, -0.055908203, 0.040771484, 0.0032806396, -0.01928711, 0.05126953, -0.030883789, -0.030395508, 0.033447266, 0.029296875, 0.009277344, 0.021118164, -0.016479492, 0.006378174, -0.006591797, -0.007537842, 0.0045166016, -0.006378174, 0.033203125, 0.018554688, -0.010986328, -0.017456055, -0.03930664, -0.018798828, 0.045898438, 0.026489258, -0.038085938, -0.007751465, -0.04321289, 0.03930664, -0.02368164, -0.00044631958, 0.00074386597, -0.029418945, 0.007537842, 0.029663086, 0.028198242, 0.017822266, -0.014099121, -0.04663086, 0.030395508, 0.007019043, 0.049560547, -0.029663086, 0.01953125, -0.004638672, 0.0

In [11]:
vector_search = text_res.text_embedding.segments[0].embeddings_float
print (vector_search)

[0.036621094, -0.0061950684, 0.018188477, -0.041992188, -0.024658203, 0.018432617, -0.01953125, 0.007507324, -0.032714844, -0.017700195, 0.014465332, -0.020019531, 0.007659912, 0.018676758, 0.029785156, -0.01965332, 0.0010299683, -0.022827148, 0.0022277832, -0.055908203, 0.040771484, 0.0032806396, -0.01928711, 0.05126953, -0.030883789, -0.030395508, 0.033447266, 0.029296875, 0.009277344, 0.021118164, -0.016479492, 0.006378174, -0.006591797, -0.007537842, 0.0045166016, -0.006378174, 0.033203125, 0.018554688, -0.010986328, -0.017456055, -0.03930664, -0.018798828, 0.045898438, 0.026489258, -0.038085938, -0.007751465, -0.04321289, 0.03930664, -0.02368164, -0.00044631958, 0.00074386597, -0.029418945, 0.007537842, 0.029663086, 0.028198242, 0.017822266, -0.014099121, -0.04663086, 0.030395508, 0.007019043, 0.049560547, -0.029663086, 0.01953125, -0.004638672, 0.009643555, 0.013671875, -0.012023926, -0.033203125, 0.04248047, 0.043701172, 0.018188477, -0.0034484863, 0.033447266, 0.04248047, 0.054

In [12]:
# Perform vector search
query_vector = vector_search

search_query = {
    "size": 5,
    "query": {
        "knn": {
            "embedding_field": {
                "vector": query_vector,
                "k": 1
            }
        }
    }
}

In [13]:
print (index_name)

twelve_labs_index


In [14]:
print (search_query)

{'size': 5, 'query': {'knn': {'embedding_field': {'vector': [0.036621094, -0.0061950684, 0.018188477, -0.041992188, -0.024658203, 0.018432617, -0.01953125, 0.007507324, -0.032714844, -0.017700195, 0.014465332, -0.020019531, 0.007659912, 0.018676758, 0.029785156, -0.01965332, 0.0010299683, -0.022827148, 0.0022277832, -0.055908203, 0.040771484, 0.0032806396, -0.01928711, 0.05126953, -0.030883789, -0.030395508, 0.033447266, 0.029296875, 0.009277344, 0.021118164, -0.016479492, 0.006378174, -0.006591797, -0.007537842, 0.0045166016, -0.006378174, 0.033203125, 0.018554688, -0.010986328, -0.017456055, -0.03930664, -0.018798828, 0.045898438, 0.026489258, -0.038085938, -0.007751465, -0.04321289, 0.03930664, -0.02368164, -0.00044631958, 0.00074386597, -0.029418945, 0.007537842, 0.029663086, 0.028198242, 0.017822266, -0.014099121, -0.04663086, 0.030395508, 0.007019043, 0.049560547, -0.029663086, 0.01953125, -0.004638672, 0.009643555, 0.013671875, -0.012023926, -0.033203125, 0.04248047, 0.043701172

In [15]:
# Execute search
response = client_aos.search(
    index=index_name,
    body=search_query
)

print (response)

# Print results
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.44936937, 'hits': [{'_index': 'twelve_labs_index', '_id': '0', '_score': 0.44936937, '_source': {'embedding_field': [0.022429451, 0.00040668788, -0.01825908, -0.005862708, -0.03371106, -6.357456e-05, -0.015320076, -0.042556215, -0.02782445, -0.00019097517, 0.03258314, -0.0061399476, -0.00049206393, 0.035632476, 0.028209884, 0.02875258, -0.035486065, -0.11288028, -0.040782217, -0.0359422, 0.015908664, -0.021092793, 0.016303983, 0.06351931, -0.03370151, -0.059598114, 0.034242813, 0.013847672, -0.0077595757, 0.02485134, -0.00448969, 0.016366754, -0.015583887, 0.040098537, 0.009583407, -0.0028939361, 0.019334463, 0.028072173, -0.025249695, -0.023403516, -0.010630837, 0.02070483, 0.026287371, -0.026809538, 0.024101667, 0.024479926, -0.015702814, 0.01550865, -0.016303834, 0.01995538, 0.019346705, 0.0451287, -0.022148034, 0.05122

## Create Audio Embeddings and Perform Audio-to-Video Search

We will work on this audio data to showcase audio to video search: [Rock 808 beat.mp3 by T_roy_920 -- https://freesound.org/s/425556/ -- License: Creative Commons 0](https://freesound.org/people/T_roy_920/sounds/425556/). We have downloaded the audio locally and uploaded it to the S3 bucket created earlier.

In [24]:
audio_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='425556__t_roy_920__rock-808-beat.mp3', Filename='audio-data.mp3')

In [25]:
# Create audio embeddings
audio_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    audio_file="audio-data.mp3"
)

print(f"Created audio embedding: model_name={audio_res.model_name}")
print(f" Model: {audio_res.model_name}")

if audio_res.audio_embedding is not None and audio_res.audio_embedding.segments is not None:
    print_segments(audio_res.audio_embedding.segments)

Created audio embedding: model_name=Marengo-retrieval-2.7
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=0.0 end_offset_sec=None
  embeddings: [-0.064453125, -0.016601562, -0.0056762695, -0.014770508, 0.012207031, -0.009216309, 0.021484375, -0.015258789, -0.0039978027, -0.0039978027, 0.030639648, -0.036376953, 0.024291992, 0.0018692017, 0.012878418, 0.041015625, 0.020019531, 0.026855469, -0.007507324, -0.021728516, -0.011108398, -0.003189087, 0.032470703, -0.007446289, 0.03149414, -0.0054626465, 0.009399414, 0.025878906, 0.034423828, -0.033935547, 0.023071289, 0.027832031, -0.016723633, -0.028564453, -0.04663086, -0.044189453, -0.0154418945, -0.015136719, 0.03466797, -0.016845703, -0.020141602, -0.01550293, -0.04345703, 0.0073547363, -0.055419922, 0.021362305, 0.055664062, 0.009643555, 0.013183594, -0.0073242188, -4.172325e-06, -0.0015411377, 0.02734375, 0.0010223389, -0.026000977, -0.008911133, -0.0050354004, -0.030151367, -0.0016403198, -0.0021362305, -0.039794

In [27]:
vector_search = audio_res.audio_embedding.segments[0].embeddings_float
print (vector_search)

[-0.064453125, -0.016601562, -0.0056762695, -0.014770508, 0.012207031, -0.009216309, 0.021484375, -0.015258789, -0.0039978027, -0.0039978027, 0.030639648, -0.036376953, 0.024291992, 0.0018692017, 0.012878418, 0.041015625, 0.020019531, 0.026855469, -0.007507324, -0.021728516, -0.011108398, -0.003189087, 0.032470703, -0.007446289, 0.03149414, -0.0054626465, 0.009399414, 0.025878906, 0.034423828, -0.033935547, 0.023071289, 0.027832031, -0.016723633, -0.028564453, -0.04663086, -0.044189453, -0.0154418945, -0.015136719, 0.03466797, -0.016845703, -0.020141602, -0.01550293, -0.04345703, 0.0073547363, -0.055419922, 0.021362305, 0.055664062, 0.009643555, 0.013183594, -0.0073242188, -4.172325e-06, -0.0015411377, 0.02734375, 0.0010223389, -0.026000977, -0.008911133, -0.0050354004, -0.030151367, -0.0016403198, -0.0021362305, -0.039794922, 0.0070495605, 0.021362305, 0.030883789, 0.011657715, 0.018188477, -0.020629883, 0.0025634766, 0.03564453, 0.0067749023, -0.024902344, -0.0043029785, -0.003814697

In [28]:
# Perform vector search
query_vector = vector_search

search_query = {
    "size": 5,
    "query": {
        "knn": {
            "embedding_field": {
                "vector": query_vector,
                "k": 1
            }
        }
    }
}

In [29]:
print (search_query)

{'size': 5, 'query': {'knn': {'embedding_field': {'vector': [-0.064453125, -0.016601562, -0.0056762695, -0.014770508, 0.012207031, -0.009216309, 0.021484375, -0.015258789, -0.0039978027, -0.0039978027, 0.030639648, -0.036376953, 0.024291992, 0.0018692017, 0.012878418, 0.041015625, 0.020019531, 0.026855469, -0.007507324, -0.021728516, -0.011108398, -0.003189087, 0.032470703, -0.007446289, 0.03149414, -0.0054626465, 0.009399414, 0.025878906, 0.034423828, -0.033935547, 0.023071289, 0.027832031, -0.016723633, -0.028564453, -0.04663086, -0.044189453, -0.0154418945, -0.015136719, 0.03466797, -0.016845703, -0.020141602, -0.01550293, -0.04345703, 0.0073547363, -0.055419922, 0.021362305, 0.055664062, 0.009643555, 0.013183594, -0.0073242188, -4.172325e-06, -0.0015411377, 0.02734375, 0.0010223389, -0.026000977, -0.008911133, -0.0050354004, -0.030151367, -0.0016403198, -0.0021362305, -0.039794922, 0.0070495605, 0.021362305, 0.030883789, 0.011657715, 0.018188477, -0.020629883, 0.0025634766, 0.03564

In [30]:
# Execute search
response = client_aos.search(
    index=index_name,
    body=search_query
)

print (response)

# Print results
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.28480396, 'hits': [{'_index': 'twelve_labs_index', '_id': '1', '_score': 0.28480396, '_source': {'embedding_field': [0.01979752, 0.004708801, -0.0017122026, -0.006176754, -0.02685851, -0.018745324, -0.03052396, -0.039830208, -0.016525451, -0.013404451, 0.026548598, -0.011071049, -0.019091459, 0.030356497, 0.029273478, 0.025740413, -0.050889157, -0.11513379, -0.035332736, -0.029131396, 0.023334717, -0.011643595, 0.03968289, 0.078003734, -0.03323544, -0.049367364, 0.025179286, -0.0012789549, -0.01347904, 0.020132825, -0.009995942, 0.00080662215, -0.012698582, 0.023589406, -0.0071470737, -0.0015084702, 0.009674214, 0.010865727, -0.029821407, -0.021768963, 0.0016050747, 0.020241138, 0.020353634, -0.040588036, 0.020285547, 0.03800083, -0.019586956, 0.015418632, -0.02124944, 0.009792225, 0.021452395, 0.05996274, -0.0041606026, 0

## Create Image Embeddings and Perform Image-to-Video Search

We will work on this image data to showcase image to video search: [Dragonfly insect image via Pixabay](https://pixabay.com/photos/dragonfly-insect-wings-nature-9326948/). We have downloaded the audio locally and uploaded it to the S3 bucket created earlier.

In [3]:
image_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='dragonfly-9326948_1280.jpg', Filename='image-data.jpg')

In [31]:
# Create image embeddings

image_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    image_file="image-data.jpg"
)

print(f"Created image embedding: model_name={image_res.model_name}")
print(f" Model: {image_res.model_name}")

if image_res.image_embedding is not None and image_res.image_embedding.segments is not None:
    print_segments(image_res.image_embedding.segments)

Created image embedding: model_name=Marengo-retrieval-2.7
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=None end_offset_sec=None
  embeddings: [0.0092454655, -0.047459874, 0.016649669, 0.012162364, -0.02010138, 0.010356426, -0.007675569, -0.018342203, -0.009956418, -0.021036027, 0.008511656, 0.0135048665, 0.03850836, -0.005519406, 0.0043635364, 0.0063705505, -0.0050345934, -0.09236061, 0.017924983, 0.004884713, 0.037156183, -0.0011020332, -0.013537122, 0.0438174, -0.014720182, -0.021849226, 0.0069117695, -0.0023453292, 0.03222177, 0.04057878, 0.0070493086, -0.011774607, 0.025694283, 0.026668768, -0.0064794472, 0.0155301355, 0.010016685, -0.0030788004, 0.0077061877, -0.00587689, -0.0278356, -0.0043119974, 0.027843762, 0.0032649196, -0.0142780505, 0.035258066, -0.049416155, 0.011853492, -0.014069501, -0.0016990341, -0.0018570074, 0.018391903, -0.012772127, 0.036828857, 0.08413021, -0.04325754, -0.049818687, 0.0030259728, 0.0077153766, -0.020328958, 0.0011359822, 0

In [33]:
vector_search = image_res.image_embedding.segments[0].embeddings_float
print (vector_search)

[0.0092454655, -0.047459874, 0.016649669, 0.012162364, -0.02010138, 0.010356426, -0.007675569, -0.018342203, -0.009956418, -0.021036027, 0.008511656, 0.0135048665, 0.03850836, -0.005519406, 0.0043635364, 0.0063705505, -0.0050345934, -0.09236061, 0.017924983, 0.004884713, 0.037156183, -0.0011020332, -0.013537122, 0.0438174, -0.014720182, -0.021849226, 0.0069117695, -0.0023453292, 0.03222177, 0.04057878, 0.0070493086, -0.011774607, 0.025694283, 0.026668768, -0.0064794472, 0.0155301355, 0.010016685, -0.0030788004, 0.0077061877, -0.00587689, -0.0278356, -0.0043119974, 0.027843762, 0.0032649196, -0.0142780505, 0.035258066, -0.049416155, 0.011853492, -0.014069501, -0.0016990341, -0.0018570074, 0.018391903, -0.012772127, 0.036828857, 0.08413021, -0.04325754, -0.049818687, 0.0030259728, 0.0077153766, -0.020328958, 0.0011359822, 0.011312732, 0.035046645, -0.010273552, -0.019929335, -0.056600757, 0.01723954, 0.020086251, -0.022728221, 0.04959231, 0.00817872, 0.039343856, -0.044556446, 0.00875635

In [34]:
# Perform vector search
query_vector = vector_search

search_query = {
    "size": 5,
    "query": {
        "knn": {
            "embedding_field": {
                "vector": query_vector,
                "k": 1
            }
        }
    }
}

In [35]:
print (search_query)

{'size': 5, 'query': {'knn': {'embedding_field': {'vector': [0.0092454655, -0.047459874, 0.016649669, 0.012162364, -0.02010138, 0.010356426, -0.007675569, -0.018342203, -0.009956418, -0.021036027, 0.008511656, 0.0135048665, 0.03850836, -0.005519406, 0.0043635364, 0.0063705505, -0.0050345934, -0.09236061, 0.017924983, 0.004884713, 0.037156183, -0.0011020332, -0.013537122, 0.0438174, -0.014720182, -0.021849226, 0.0069117695, -0.0023453292, 0.03222177, 0.04057878, 0.0070493086, -0.011774607, 0.025694283, 0.026668768, -0.0064794472, 0.0155301355, 0.010016685, -0.0030788004, 0.0077061877, -0.00587689, -0.0278356, -0.0043119974, 0.027843762, 0.0032649196, -0.0142780505, 0.035258066, -0.049416155, 0.011853492, -0.014069501, -0.0016990341, -0.0018570074, 0.018391903, -0.012772127, 0.036828857, 0.08413021, -0.04325754, -0.049818687, 0.0030259728, 0.0077153766, -0.020328958, 0.0011359822, 0.011312732, 0.035046645, -0.010273552, -0.019929335, -0.056600757, 0.01723954, 0.020086251, -0.022728221, 0

In [36]:
# Execute search
response = client_aos.search(
    index=index_name,
    body=search_query
)

print (response)

# Print results
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 0.57250696, 'hits': [{'_index': 'twelve_labs_index', '_id': '1', '_score': 0.57250696, '_source': {'embedding_field': [0.01979752, 0.004708801, -0.0017122026, -0.006176754, -0.02685851, -0.018745324, -0.03052396, -0.039830208, -0.016525451, -0.013404451, 0.026548598, -0.011071049, -0.019091459, 0.030356497, 0.029273478, 0.025740413, -0.050889157, -0.11513379, -0.035332736, -0.029131396, 0.023334717, -0.011643595, 0.03968289, 0.078003734, -0.03323544, -0.049367364, 0.025179286, -0.0012789549, -0.01347904, 0.020132825, -0.009995942, 0.00080662215, -0.012698582, 0.023589406, -0.0071470737, -0.0015084702, 0.009674214, 0.010865727, -0.029821407, -0.021768963, 0.0016050747, 0.020241138, 0.020353634, -0.040588036, 0.020285547, 0.03800083, -0.019586956, 0.015418632, -0.02124944, 0.009792225, 0.021452395, 0.05996274, -0.0041606026, 0